In [1]:
import os

In [2]:
os.getcwd()

'e:\\Projects\\Chicken-Coccidiosis-Classification-Project\\research'

In [3]:
%pwd

'e:\\Projects\\Chicken-Coccidiosis-Classification-Project\\research'

In [4]:
os.chdir('../')

In [32]:
%pwd

'e:\\Projects\\Chicken-Coccidiosis-Classification-Project'

In [33]:
from dataclasses import dataclass
from pathlib import Path

In [34]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [35]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [36]:
config = read_yaml(CONFIG_FILE_PATH)
config

[2024-01-04 15:37:46,960:INFO:common:yaml file: config\config.yaml loaded successfully]


ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}})

In [37]:
config.data_ingestion.root_dir

'artifacts/data_ingestion'

In [38]:
dic = DataIngestionConfig(
    root_dir=config.data_ingestion.root_dir,
    source_URL=config.data_ingestion.source_URL,
    local_data_file=config.data_ingestion.local_data_file,
    unzip_dir=config.data_ingestion.unzip_dir
)

In [39]:
dic

DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://github.com/entbappy/Branching-tutorial/raw/master/Chicken-fecal-images.zip', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')

In [12]:
read_yaml(PARAMS_FILE_PATH)

[2024-01-04 14:51:36,547:INFO:common:yaml file: params.yaml loaded successfully]


ConfigBox({'key': 'val'})

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [40]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [42]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f'{filename} download! with following info: \n{headers}')
        else:
            logger.info(f'File already exists of size: {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [43]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-04 15:55:07,688:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-01-04 15:55:07,689:INFO:common:yaml file: params.yaml loaded successfully]
[2024-01-04 15:55:07,689:INFO:common:created directory at: artifacts]
[2024-01-04 15:55:07,695:INFO:common:created directory at: artifacts/data_ingestion]
[2024-01-04 15:55:12,076:INFO:1419213275:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1032:1DC3B6:17CBF9:25F624:65968783
Accept-Ranges: bytes
Date: Thu, 04 Jan 2024 10:25:09 GMT
Via: 1.1 varnish
X-Served-By: cache-del21733-DEL
X-Cache: MISS
X-Cache-Hi